In [8]:
import sys 
sys.path.append('./utils')
from amg_utils import *

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

In [15]:
check_table(name="PRODUCTION.BD_S2CINTERNAL.V_CM_TONIC_KEYWORD_DATA")

,,,,,,,
0,eventdate,object,2023-03-23,2023-03-23,2023-03-23,2023-03-23,2023-03-23
1,fb_account,float64,NaN,1626256331047204.0,NaN,NaN,NaN
2,fb_campaign_name,object,nail_fungus-fb-71-d2l-mob-dn,Age_Spot_Treatment-fb-71-d2l-mob-dn,kidney-fb-56-d2l-mob-ja,hair_transplant-fb-56-d2l-mob-ja,colon_cancer-fb-71-d2l-mob-dn
3,fb_adset_name,object,nail_fungus-adset 1,Age_Spot_Treatment-adset 1,kidney_d2l_056_adset02,hair_transplant_d2l_056_adset01,colon_cancer-adset 1
4,tonic_campaign_name,object,fb-us-health-nail_fungus_treatment,fb-us-health-Age_Spots,fb-us-health-kidney-2,fb-us-beauty-hair_transplantation,fb-us-health-colon_cancer
5,media_buyer_name,object,Darren Nicastro,Darren Nicastro,Jordan Allen,Jordan Allen,Darren Nicastro
6,keyword,object,Homeo Remedy,Cream For Dark Spot Remover,Chronic Kidney Disease Diet Food List,Hair Transplant Cost,Early Signs Of Colon Cancer And Treatments
7,clicks,int64,1,1,1,1,1
8,actual_net_revenue,float64,0.05,0.29,0.05,3.23,0.07
9,rpc,float64,0.05,0.29,0.05,3.23,0.07


In [13]:
gn_kws_query = """

with c as (
    select 
        *,
        row_number() over (partition by id order by updated desc nulls last) as rn
    from SEM_TOOLS.CM_FB_CAMPAIGN_MANAGEMENT.TONIC_CAMPAIGN
    qualify rn = 1
), k as (
    select 
        fb_account,
        fb_campaign_name,
        fb_adset_name,
        tonic_campaign_name,
        keyword,
        sum(clicks) as clicks,
        sum(actual_net_revenue) as actual_net_revenue,
        sum(actual_net_revenue) / sum(clicks) as rpc
    from PRODUCTION.BD_S2CINTERNAL.V_CM_TONIC_KEYWORD_DATA
    where eventdate >= dateadd(DAY, -14, CURRENT_DATE::DATE)
    group by 1,2,3,4,5
), p as (
    select 
        account_id,
        campaign_name, 
        adset_name,
        sum(fb_spend) as spend,
        sum(net_revenue) as rev,
        rev - spend as contrib,
        sum(rev_clicks) as rev_clicks
    from production.bd_s2cinternal.v_cm_fb_tonic_daily_combine
    where eventdate >= dateadd(DAY, -14, CURRENT_DATE::DATE)
    group by 1,2,3
)
select
    campaign_name,
    adset_name,
    tonic_campaign_name,
    spend,
    rev,
    contrib,
    rev_clicks,
    keyword,
    clicks,
    actual_net_revenue,
    rpc,
    c.country_code,
    c.offer_id
from p 
join k on p.account_id = k.fb_account and p.campaign_name = k.fb_campaign_name and p.adset_name = k.fb_adset_name
join c on k.tonic_campaign_name = c.name
where c.country_code = 'US'
  and c.status = 'active'
  and contrib > 10
order by campaign_name, adset_name, actual_net_revenue desc;
"""


gn_kws = eq(gn_kws_query)

peek(gn_kws)

\
0         campaign_name   object     All_Inclusive_Cancun_US-fb-111-d2l-mob   
1            adset_name   object        All_Inclusive_Cancun_dbc_US-adset 1   
2   tonic_campaign_name   object       fb-US-Travel-Cancun-All-Inclusive_v1   
3                 spend  float64                                     563.07   
4                   rev  float64                                 626.146422   
5               contrib  float64                                  63.076422   
6            rev_clicks    int64                                       1938   
7               keyword   object  Cancun All Inclusive Packages With Flight   
8                clicks    int64                                       1445   
9    actual_net_revenue  float64                                 452.118676   
10                  rpc  float64                                   0.312885   
11         country_code   object                                         US   
12             offer_id   object                                        325   

                                            \
0   All_Inclusive_Cancun_US-fb-111-d2l-mob   
1      All_Inclusive_Cancun_dbc_US-adset 1   
2     fb-US-Travel-Cancun-All-Inclusive_v1   
3                                   563.07   
4                               626.146422   
5                                63.076422   
6                                     1938   
7         Cancun All Inclusive Adults Only   
8                                      254   
9                                83.692072   
10                                0.329496   
11                                      US   
12                                     325   

                                            \
0   All_Inclusive_Cancun_US-fb-111-d2l-mob   
1      All_Inclusive_Cancun_dbc_US-adset 1   
2     fb-US-Travel-Cancun-All-Inclusive_v1   
3                                   563.07   
4                               626.146422   
5                                63.076422   
6                                     1938   
7            Cancun All Inclusive Packages   
8                                      140   
9                                60.245942   
10                                0.430328   
11                                      US   
12                                     325   

                                            \
0   All_Inclusive_Cancun_US-fb-111-d2l-mob   
1      All_Inclusive_Cancun_dbc_US-adset 1   
2     fb-US-Travel-Cancun-All-Inclusive_v1   
3                                   563.07   
4                               626.146422   
5                                63.076422   
6                                     1938   
7                  Cheap Vacation Packages   
8                                       25   
9                                14.318333   
10                                0.572733   
11                                      US   
12                                     325   

                                            \
0   All_Inclusive_Cancun_US-fb-111-d2l-mob   
1      All_Inclusive_Cancun_dbc_US-adset 1   
2     fb-US-Travel-Cancun-All-Inclusive_v1   
3                                   563.07   
4                               626.146422   
5                                63.076422   
6                                     1938   
7             Cancun All Inclusive Resorts   
8                                       34   
9                                   7.1854   
10                                0.211335   
11                                      US   
12                                     325   

                                            \
0   All_Inclusive_Cancun_US-fb-111-d2l-mob   
1      All_Inclusive_Cancun_dbc_US-adset 1   
2     fb-US-Travel-Cancun-All-Inclusive_v1   
3                                   563.07   
4                               626.146422   
5                                63.076422   
6                                     1938   
7                     

In [ ]:

peek(eq("""WITH ads AS (
    SELECT 
        id AS adset_id,
        status,
        budget,
        budget_type,
        fb_created,
        DATE(updated) AS date_updated        
    FROM (
        SELECT 
            *,        
            ROW_NUMBER() OVER(PARTITION BY id, DATE(updated) ORDER BY updated ASC) AS rn_asc
        FROM SEM_TOOLS.cm_fb_campaign_management.ad_set
        where id = 120203732421540410
    ) subquery
    WHERE rn_asc = 1
)"""))


In [10]:
peek(eq("select * from SEM_TOOLS.cm_fb_campaign_management.ad_set where id = 120203732422100410 order by updated"))

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,id,int64,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120

In [12]:
peek(eq("select * from PRODUCTION.BD_S2CINTERNAL.v_cm_fb_tonic_daily_combine where adset_id = 120203732421540410 order by eventdate"))

,,,,,,,,,,,,
0,eventdate,object,2024-01-17,2024-01-18,2024-01-19,2024-01-20,2024-01-21,2024-01-22,2024-01-23,2024-01-24,2024-01-25,2024-01-26
1,buyer_initials,object,GN,GN,GN,GN,GN,GN,GN,GN,GN,GN
2,buyer_name,object,Golden Nuggets,Golden Nuggets,Golden Nuggets,Golden Nuggets,Golden Nuggets,Golden Nuggets,Golden Nuggets,Golden Nuggets,Golden Nuggets,Golden Nuggets
3,account_currency,object,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD
4,account_id,int64,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781
5,account_name,object,fw84 - FB SCMT 1016 GN (America/Los_Angeles),fw84 - FB SCMT 1016 GN (America/Los_Angeles),fw84 - FB SCMT 1016 GN (America/Los_Angeles),fw84 - FB SCMT 1016 GN (America/Los_Angeles),fw84 - FB SCMT 1016 GN (America/Los_Angeles),fw84 - FB SCMT 1016 GN (America/Los_Angeles),fw84 - FB SCMT 1016 GN (America/Los_Angeles),fw84 - FB SCMT 1016 GN (America/Los_Angeles),fw84 - FB SCMT 1016 GN (America/Los_Angeles),fw84 - FB SCMT 1016 GN (America/Los_Angeles)
6,ad_id,int64,120203732424650410,120203732424650410,120203732424650410,120203732424650410,120203732424650410,120203732424650410,120203732424650410,120203732424650410,120203732424650410,120203732424650410
7,ad_name,object,batch_01|TEST_5|20240117|Ad1,batch_01|TEST_5|20240117|Ad1,batch_01|TEST_5|20240117|Ad1,batch_01|TEST_5|20240117|Ad1,batch_01|TEST_5|20240117|Ad1,batch_01|TEST_5|20240117|Ad1,batch_01|TEST_5|20240117|Ad1,batch_01|TEST_5|20240117|Ad1,batch_01|TEST_5|20240117|Ad1,batch_01|TEST_5|20240117|Ad1
8,adset_id,int64,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410,120203732422100410
9,adset_name,object,batch_01|TEST_5|20240117|Adset1,batch_01|TEST_5|20240117|Adset1,batch_01|TEST_5|20240117|Adset1,batch_01|TEST_5|20240117|Adset1,batch_01|TEST_5|20240117|Adset1,batch_01|TEST_5|20240117|Adset1,batch_01|TEST_5|20240117|Adset1,batch_01|TEST_5|20240117|Adset1,batch_01|TEST_5|20240117|Adset1,batch_01|TEST_5|20240117|Adset1
